In [2]:
# Setup
# `jupyter nbextension enable --py --sys-prefix ipyvuetify`#
# start: `voila .\smart_grids.ipynb`
# icons here https://vuetifyjs.com/en/features/icon-fonts/#material-design-icons


In [1]:
!

^C


In [3]:
import json
import pathlib

import ipyvuetify as v
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import requests
from ipywidgets import jslink

from util import get_photovoltaic_data, get_coordinates

PTV_API_KEY = pathlib.Path("key.txt").read_text()
BATTERIES_FILE = "batteries.json"


In [4]:
def get_data_fig(output):
    return go.Figure(
        data=[go.Bar(x=[o["time"] for o in output], y=[o["G(i)"] for o in output])],
        layout_title_text="A Figure Displaying Itself",
    )


# {'time': '20150101:0010', 'G(i)': 0.0, 'H_sun': 0.0, 'T2m': -9.23, 'WS10m': 0.5, 'Int': 0.0}
# G(i): Global irradiance on the inclined plane (plane of the array) (W/m2)
# H_sun: Sun height (degree)
# T2m: 2-m air temperature (degree Celsius)
# WS10m: 10-m total wind speed (m/s)
# Int: 1 means solar radiation values are reconstructed


In [147]:
# Address
address_title = v.Subheader(children=["Address"])

country = v.TextField(label="Land", v_model="", disabled=True)
city = v.TextField(label="Stadt", v_model="", disabled=True)
latitude = v.TextField(label="Latitude", v_model="", disabled=True)
longitude = v.TextField(label="Longitude", v_model="", disabled=True)
address_row = v.Col(
    children=[
        v.Row(children=[country]),
        v.Row(children=[city]),
        v.Row(children=[latitude]),
        v.Row(children=[longitude]),
    ],
)

search_address_text_field = v.TextField(
    label="Adresse", v_model="", prepend_inner_icon="mdi-map-marker"
)


def get_address(widget, event, data):
    address_card.loading = True
    coordinates = get_coordinates.get_coordinates(PTV_API_KEY, widget.v_model)
    country.v_model = coordinates["country"]
    city.v_model = coordinates["city"]
    latitude.v_model = coordinates["latitude"]
    longitude.v_model = coordinates["longitude"]
    address_card.loading = False
search_address_text_field.on_event("change", get_address)

address_card = v.Card(
    loading=False,
    elevation=0,
    children=[
        v.Row(children=[address_title]),
        v.Row(
            children=[
                v.Col(cols=7,
                    children=[search_address_text_field],
                ),
                v.Col(cols=5,
                    children=[address_row],
                ),
            ]
        ),
    ],
)
v.Container(children=[address_card, v.Divider()])

# get photovoltaic data
# data = get_photovoltaic_data(coordinates["latitude"],coordinates["longitude"])
# data_text.v_model = (
#     f'{data}'
# )


Container(children=[Card(children=[Row(children=[Subheader(children=['Address'])]), Row(children=[Col(children…

In [146]:
# Battery
battery_df = pd.read_json(BATTERIES_FILE).set_index("id").sort_values("efficiency_rank")
battery_names = list(battery_df["name"])
# battery_df.head(2)

battery_title = v.Subheader(children=["Battery"])

battery_ranking = v.TextField(label="Efficiency ranking", v_model="", disabled=True)
battery_type = v.TextField(label="Type", v_model="", disabled=True)
battery_capacity = v.TextField(label="Capacity", v_model="", disabled=True)
battery_average_efficiency = v.TextField(
    label="Average efficiency", v_model="", disabled=True
)

battery_row = v.Col(
    children=[
        v.Row(children=[battery_ranking]),
        v.Row(children=[battery_type]),
        v.Row(children=[battery_capacity]),
        v.Row(children=[battery_average_efficiency]),
    ],
)
battery_select = v.Select(
    label="Battery", v_model=battery_names[0], items=battery_names
)


def battery_change(widget, event, data):
    battery_card.loading = True
    battery = battery_df.loc[battery_df["name"] == widget.v_model]
    battery_ranking.v_model = str(battery["efficiency_rank"].values[0])
    battery_type.v_model = str(battery["type"].values[0])
    battery_capacity.v_model = str(battery["capacity"].values[0])
    battery_average_efficiency.v_model = str(battery["average_efficiency"].values[0])
    battery_card.loading = False


battery_select.on_event("change", battery_change)

battery_card = v.Card(
    loading=False,
    elevation=0,
    children=[
        v.Row(children=[battery_title]),
        v.Row(
            children=[
                v.Col(cols = 7,
                    children=[battery_select],
                ),
                v.Col(cols=5,
                    children=[battery_row],
                ),
            ]
        ),
    ],
)
v.Container(children=[battery_card,v.Divider()])
# get photovoltaic data
# data = get_photovoltaic_data(coordinates["latitude"],coordinates["longitude"])
# data_text.v_model = (
#     f'{data}'
# )


Container(children=[Card(children=[Row(children=[Subheader(children=['Battery'])]), Row(children=[Col(children…

In [120]:
# PV
pv_title = v.Subheader(children=["Photovoltaic"])

pv_peak_power = v.TextField(
    label="Installed peak power in Wp",
    reverse=True,
    type="number",
    v_model="1000",
)
pv_slope = v.Slider(v_model=45, max=90, min=0)
pv_slope_text = v.TextField(
    label="Slope in °",
    v_model="",
    type="number",
    reverse=True,
    style_="text_align:right;",
)
jslink((pv_slope, "v_model"), (pv_slope_text, "v_model"))

pv_azimuth = v.Slider(
    v_model=180,
    max=359,
    min=0,
)
pv_azimuth_text = v.TextField(
    label="Azimuth in °", v_model="", type="number", reverse=True
)
jslink((pv_azimuth, "v_model"), (pv_azimuth_text, "v_model"))

pv_card = v.Card(
    elevation=0,
    children=[
        v.Row(children=[pv_title]),
        v.Row(children=[v.Col(cols=3, children=[pv_peak_power]), v.Col(cols=9)]),
        v.Row(
            children=[
                v.Col(cols=3, children=[pv_slope_text]),
                v.Col(cols=9, children=[pv_slope]),
            ]
        ),
        v.Row(
            children=[
                v.Col(cols=3, children=[pv_azimuth_text]),
                v.Col(cols=9, children=[pv_azimuth]),
            ]
        ),
    ],
)
v.Container(children=[pv_card, v.Divider()])


Container(children=[Card(children=[Row(children=[Subheader(children=['Photovoltaic'])]), Row(children=[Col(chi…

In [142]:
# Consumption
consumption_title = v.Subheader(children=["Consumption"])
consumer_name = v.TextField(label="Name", v_model="")
consumer_usage = v.TextField(label="Usage", v_model="")
consumer_create_btn = v.Btn(children=[v.Icon(children=["mdi-plus"])])
consumer_row = v.Row(
    children=[v.Col(children=[consumer_name]), v.Col(children=[consumer_usage]), v.Col(children=[consumer_create_btn])],
)
consumer_list = [v.TextField(label="name")]

consumption_card = v.Card(
    children=[v.Row(children=[consumption_title])],
)
v.Container(
    children=[
        v.Row(children=[consumption_card]),
        consumer_row,
    ]
)


Container(children=[Row(children=[Card(children=[Row(children=[Subheader(children=['Consumption'])])])]), Row(…